<h1 align="center" style="background-color:#616161;color:white">Training and Test Set Selection</h1>

There will be two types of assessments: 
* Predicting what and when known users would like to listen to next
* Predicting what and when previously unknown users would like to listen to next

<b>Predicting next action of a known user:</b>
* For every user 3 randomly selected time slots will be chosen from the last 10% of their listening history
    * A timeslot in the 10-30 minute interval range. This is to assess
    * A timeslot in the 12 hours+ interval range
    * A timeslot in the 10minutes + interval range. This is to act as a control sample to assess whether the interval range do make a difference in prediction.
    
<b>Unknown users sample:</b>
* 10 users will be randomly selected and separated out to be used in evaluating unknown users

<h3 style="background-color:#616161;color:white">Setup code (you can ignore this)</h3>

In [1]:
# import modules
%matplotlib inline
import os
import sys
import numpy as np
import pandas as pd
import datetime
import csv
import json
import sqlite3
from pathlib import Path
from sklearn.manifold import TSNE
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt
import gensim, logging

## Parameters you can change

# Abs path to settings file
#root = "C:/DS/Github/MusicRecommendation"  # BA, Windows
root = "/home/badrul/Documents/github/MusicRecommendation" # BA, Linux

## Import the codebase module
fPath = root + "/1_codemodule"
if fPath not in sys.path: sys.path.append(fPath)

import coreCode as cb
import lastfmCode as fm

## Finish setting up
os.chdir(root)

settingsDict =  cb.loadSettings()

dbPath = root + settingsDict['mainDbPath']
fmSimilarDbPath = root + settingsDict['fmSimilarDbPath']
fmTagsDbPath = root + settingsDict['fmTagsDbPath']
trackMetaDbPath = root + settingsDict['trackmetadata']


<h3 style="background-color:#616161;color:white">Select random users</h3>

In [9]:
TestUsers = 10
sqlStr= "SELECT * FROM tblAgg_User WHERE UserID IN (SELECT UserID FROM tblAgg_User ORDER BY RANDOM() LIMIT {})".format(TestUsers+1)
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect(dbPath)
df = pd.read_sql_query(sqlStr, con)
con.close()

df

,userID,FirstPlay
0,148,2008-09-19 08:24:55
1,195,2007-08-01 22:53:28
2,367,2007-08-23 17:46:47
3,511,2005-12-14 00:50:27
4,622,2006-06-03 02:57:04
5,685,2005-12-09 22:23:10
6,699,2007-03-03 05:35:05
7,747,2006-04-21 08:48:32
8,847,2006-01-02 23:03:03
9,872,2008-02-19 19:27:38


In [17]:
# Store the record of which 10 users were selected
    
testUsers = [148,195,367,511,622,685,747,847,872,894]
con = sqlite3.connect(dbPath)

for UserID in testUsers:
    sqlStr = "Update tblAgg_User Set TestUser = 1 where UserID = {}".format(UserID)    
    con.execute(sqlStr)
    con.commit()

con.close()

<h3 style="background-color:#616161;color:white">Select random timeslots</h3>

In [49]:
sqlStr = 'Select UserID, round(max(HistoryID)*0.9) as cutOff from tblMain Group by UserID'
con = sqlite3.connect(dbPath)
df = pd.read_sql_query(sqlStr, con)

for index, row in df.iterrows():
    c = int(row['cutOff'])
    u= row['UserID']
    sqlStr = 'Update tblAgg_User Set TestHistID = {} Where UserID = {}'.format(c,u)
    con.execute(sqlStr)

con.commit()
con.close()
